#### 1. INSTALL AND IMPORT DEPENDENCIES

In [1]:
import tensorflow as tf
import cv2,labelme
import shutil
from pathlib import Path
import os,uuid
import matplotlib.pyplot as plt
import time,json,wget
from sklearn.model_selection import train_test_split
import numpy as np,pandas as pd
import albumentations as A
import mediapipe as mp
%matplotlib inline

#### 2. CREATE PROJECT DIRECTORY

In [2]:
CUSTOM_MODEL_NAME = 'my_ssd_mobnet'
PRETRAINED_MODEL_NAME = 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8'
PRETRAINED_MODEL_URL = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz'
TF_RECORD_SCRIPT_NAME = 'generate_tfrecord_labelme.py'
LABEL_MAP_NAME = 'label_map.pbtxt'

In [3]:
PATHS = {
    'workspace':Path('workspace'),
    'images':Path('workspace','images'),
    'train-images':Path('workspace','images','train'),
    'test-images':Path('workspace','images','test'),
    'protoc': Path('workspace','protoc'),
    'scripts': Path('workspace','scripts'),
    'pretrained-model':Path('workspace','pretrained_model'),
    'annotation':Path('workspace','annotation'),
    'models': Path('workspace','models'),
    'CHECKPOINT_PATH': Path('workspace','models',CUSTOM_MODEL_NAME),
    'OUTPUT_PATH': Path('workspace','models',CUSTOM_MODEL_NAME,'export'),
    'TFJS_PATH': Path('workspace','models',CUSTOM_MODEL_NAME,'tfjsexport'),
    'TFLITE_PATH': Path('workspace','models',CUSTOM_MODEL_NAME,'tfliteexport'),
}

for key in PATHS.keys():
    PATHS[key].mkdir(exist_ok=True)

In [6]:
files = {
    'TF_RECORD_SCRIPTS': Path(PATHS['scripts'],TF_RECORD_SCRIPT_NAME),
    'LABELMAP': Path(PATHS['annotation'],LABEL_MAP_NAME),
    'PIPELINE_PATH':os.path.join(str(PATHS['pretrained-model']),'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8','pipeline.config'),
    'CHECKPOINT_PATH': os.path.join(str(PATHS['pretrained-model']),'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8','checkpoint')
}

#### 3. GET IMAGES FROM VIDEOS

In [25]:
def extract_images(video_path,output_folder,frame_interval_seconds):
    '''
    extracts images from video
    
    Arguments: 
        video_path: this is the path to the video we are extracting image from
        output_file: this is the directory our extracted image will be saved to
        frame_interval_seconds: this is the seconds interval to save each image
    '''

    cap = cv2.VideoCapture(video_path)
    frame_count = 0

    Path(output_folder).mkdir(exist_ok=True)

    while cap.isOpened():
        ret,frame = cap.read()
        if not ret:
            break

        frame_count += 1
        current_time_seconds = frame_count / cap.get(cv2.CAP_PROP_FPS)

        # Save the frame if it's within the desired interval
        if current_time_seconds % frame_interval_seconds == 0:
            output_path = Path(output_folder, f"{str(uuid.uuid1())}.jpg")
            print(output_path)
            cv2.imwrite(str(output_path), frame)
        
        

    cap.release()

In [ ]:
for vid in Path('videos').glob('*.mov'):
    extract_images(str(vid),'shot_detection_images',2)

In [22]:
!labelme

[INFO   ] __init__:get_config:70 - Loading config file from: C:\Users\bkj\.labelmerc


#### 4. SPLIT IMAGES TO TRAIN AND TEST

In [23]:
image_list = [x for x in Path('images/collectimages').glob('*.jpg')]
image_list

[WindowsPath('images/collectimages/04a91c7a-881e-11ee-b036-ec5c68664d70.jpg'),
 WindowsPath('images/collectimages/0d36237d-881e-11ee-83bf-ec5c68664d70.jpg'),
 WindowsPath('images/collectimages/16a48cf7-881e-11ee-ae78-ec5c68664d70.jpg'),
 WindowsPath('images/collectimages/1de4ef4b-881e-11ee-af5e-ec5c68664d70.jpg'),
 WindowsPath('images/collectimages/2114ab96-881e-11ee-8dc8-ec5c68664d70.jpg'),
 WindowsPath('images/collectimages/224d0618-881e-11ee-b25e-ec5c68664d70.jpg'),
 WindowsPath('images/collectimages/2389a375-881e-11ee-826e-ec5c68664d70.jpg'),
 WindowsPath('images/collectimages/24d29df5-881e-11ee-a618-ec5c68664d70.jpg'),
 WindowsPath('images/collectimages/26074d2f-881e-11ee-8429-ec5c68664d70.jpg'),
 WindowsPath('images/collectimages/274328ba-881e-11ee-bf5a-ec5c68664d70.jpg'),
 WindowsPath('images/collectimages/2895240a-881e-11ee-b2a8-ec5c68664d70.jpg'),
 WindowsPath('images/collectimages/29cc1d78-881e-11ee-b7ee-ec5c68664d70.jpg'),
 WindowsPath('images/collectimages/2b11bcd1-881e-11e

In [37]:
train,test = train_test_split(image_list,test_size = 0.2, random_state=42)

for imagelist in ['train','test']:
    if imagelist == 'train':
        for file in train:
            shutil.copy(file,PATHS['train-images'])
            label = PATHS['annotation'] / Path(file.name).with_suffix('.json')
            shutil.copy(label,PATHS['train-images'])
    else:
        for file in test:
            shutil.copy(file,PATHS['test-images'])
            label = PATHS['annotation'] / Path(file.name).with_suffix('.json')
            shutil.copy(label,PATHS['test-images'])


##### 5. SETUP TENSORFLOW OBJECT DETECTION

In [97]:
!cd {PATHS['tfod']} && git clone https://github.com/tensorflow/models .

Cloning into '.'...
Updating files:  19% (729/3668)
Updating files:  20% (734/3668)
Updating files:  21% (771/3668)
Updating files:  22% (807/3668)

In [112]:
!set PYTHONPATH=%PYTHONPATH%;C:\Users\bkj\Documents\GitHub\projects\Upwork\nba-computer-vision\tfod\research;C:\Users\bkj\Documents\GitHub\projects\Upwork\nba-computer-vision\tfod\research\slim

In [113]:
!pip install -r tfod/official/requirements.txt

DEPRECATION: Loading egg at c:\users\bkj\documents\github\projects\computer vision\cv-env\lib\site-packages\apache_beam-2.51.0rc1-py3.11-win-amd64.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..


DEPRECATION: Loading egg at c:\users\bkj\documents\github\projects\computer vision\cv-env\lib\site-packages\avro_python3-1.10.2-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..


DEPRECATION: Loading egg at c:\users\bkj\documents\github\projects\computer vision\cv-env\lib\site-packages\lvis-0.5.3-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..

DEPRECATION: Loading egg at c:\users\bkj\documents\github\projects\computer vision\cv-env\lib\site-packages\object_detection-0.1-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..

DEPRECATION: Loading egg at c:\users\bkj\documents\github\projects\computer vision\cv-env\lib\site-packages\pyparsing-2.4.7-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..


...


DEPRECATION: Loading egg at c:\users\bkj\documents\github\projects\computer vision\cv-env\lib\site-packages\tensorflow-2.13.1-py3.11-win-amd64.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..

DEPRECATION: Loading egg at c:\users\bkj\documents\github\projects\computer vision\cv-env\lib\site-packages\tensorflow_io-0.31.0-py3.11-win-amd64.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..

DEPRECATION: Loading egg at c:\users\bkj\documents\github\projects\computer vision\cv-env\lib\site-packages\tf_models_official-2.13.2-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..



[notice] A new release of pip is available: 23.2.1 -> 23.3.1

[notice] To update, run: python.exe -m pip install --upgrade pip

##### 6 DOWNLOAD PROTOC.ZIP

In [3]:
wget.download('https://github.com/protocolbuffers/protobuf/releases/download/v25.1/protoc-25.1-win64.zip')


'protoc-25.1-win64 (1).zip'

In [4]:
!copy protoc-25.1-win64.zip {PATHS['protoc']}
!cd {PATHS['protoc']} && tar -xf protoc-25.1-win64.zip
os.environ['PATH'] += os.pathsep + os.path.abspath(os.path.join(PATHS['protoc'],'bin'))  


        1 file(s) copied.


In [5]:
!cd tfod/research && protoc object_detection/protos/*.proto --python_out=. && copy object_detection\\packages\\tf2\\setup.py setup.py && python setup.py build && python setup.py install

zip_safe flag not set; analyzing archive contents...

        1 file(s) copied.

object_detection.core.__pycache__.densepose_ops.cpython-310: module references __file__


running build

object_detection.core.__pycache__.preprocessor.cpython-310: module MAY be using inspect.stack

...

copying object_detection\protos\anchor_generator_pb2.py -> build\lib\object_detection\protos



error: Could not find suitable distribution for Requirement.parse('tensorflow-text~=2.12.0')

copying object_detection\protos\argmax_matcher_pb2.py -> build\lib\object_detection\protos


copying object_detection\protos\bipartite_matcher_pb2.py -> build\lib\object_detection\protos
copying object_detection\protos\box_coder_pb2.py -> build\lib\object_detection\protos


copying object_detection\protos\box_predictor_pb2.py -> build\lib\object_detection\protos
copying object_detection\protos\calibration_pb2.py -> build\lib\object_detection\protos
copying object_detection\protos\center_net_pb2.py -> build\lib\object_detection\protos
copying object_detection\protos\eval_pb2.py -> build\lib\object_detection\protos
copying object_detection\protos\faster_rcnn_box_coder_pb2.py -> build\lib\object_detection\protos
copying object_detection\protos\faster_rcnn_pb2.py -> build\lib\object_detection\protos
copying object_detection\protos\flexible_grid_anchor_generator_pb2.py -> build\lib\object_detection\protos
copying object_detection\protos\fpn_pb2.py -> build\lib\object_detection\protos
copying object_detection\protos\graph_rewriter_pb2.py -> build\lib\object_detection\protos
copying object_detection\protos\grid_anchor_generator_pb2.py -> build\lib\object_detection\protos
copying object_detection\protos\hyperparams_pb2.py -> build\lib\object_detection\protos
cop

In [6]:
!cd tfod/research/slim && pip install -e .

Obtaining file:///C:/Users/bkj/Documents/GitHub/projects/Upwork/nba-computer-vision/tfod/research/slim


  Preparing metadata (setup.py): started


  Preparing metadata (setup.py): finished with status 'done'

  Running setup.py develop for slim



[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


##### 7. VERIFICATION SCRIPT

In [7]:
VERIFICATION_SCRIPT = os.path.join('tfod','research','object_detection','builders','model_builder_tf2_test.py')

!python {VERIFICATION_SCRIPT}

c:\Users\bkj\AppData\Local\Programs\Python\Python310\lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
Running tests under Python 3.10.2: c:\Users\bkj\AppData\Local\Programs\Python\Python310\python.exe
[ RUN      ] ModelBuilderTF2Test.test_create_center_net_deepmac
W1121 20:54:12.962249 14236 batch_normalization.py:1426] `tf.keras.layers.experimental.SyncBatchNormalization` endpoint is deprecated and will be removed in a future release. Please use `tf.keras.layers.BatchNormalization` with parameter `synchronized` set to True.
W1121 20:54:12.979268 1


W1121 20:54:12.979268 14236 batch_normalization.py:1426] `tf.keras.layers.experimental.SyncBatchNormalization` endpoint is deprecated and will be removed in a future release. Please use `tf.keras.layers.BatchNormalization` with parameter `synchronized` set to True.
W1121 20:54:13.001231 14236 batch_normalization.py:1426] `tf.keras.layers.experimental.SyncBatchNormalization` endpoint is deprecated and will be removed in a future release. Please use `tf.keras.layers.BatchNormalization` with parameter `synchronized` set to True.
W1121 20:54:13.004228 14236 batch_normalization.py:1426] `tf.keras.layers.experimental.SyncBatchNormalization` endpoint is deprecated and will be removed in a future release. Please use `tf.keras.layers.BatchNormalization` with parameter `synchronized` set to True.
W1121 20:54:13.009225 14236 batch_normalization.py:1426] `tf.keras.layers.experimental.SyncBatchNormalization` endpoint is deprecated and will be removed in a future release. Please use `tf.keras.layer

##### 8. GET PRETRAINED MODEL

In [18]:
if os.name == 'nt':
    wget.download(PRETRAINED_MODEL_URL)
    !move {PRETRAINED_MODEL_NAME + '.tar.gz'} {PATHS['custom-model']}
    !cd {PATHS['custom-model']} && tar -zxvf {PRETRAINED_MODEL_NAME + '.tar.gz'}

        1 file(s) moved.


x ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/
x ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/
x ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0.data-00000-of-00001
x ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/checkpoint
x ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0.index
x ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config
x ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/saved_model/
x ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/saved_model/saved_model.pb
x ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/saved_model/variables/
x ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/saved_model/variables/variables.data-00000-of-00001
x ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/saved_model/variables/variables.index


#### 9. CREATE LABEL MAP

In [23]:
labels = [{'name':'player','id':1}]

with open(files['LABELMAP'],'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')


In [11]:
files['TF_RECORD_SCRIPTS']

WindowsPath('workspace/scripts/generate_tfrecord_labelme.py')

In [46]:
!python "workspace/scripts/generate_tfrecord_labelme.py" -j "workspace/images/test/" -i "workspace/images/test/" -l "workspace/annotation/label_map.pbtxt" -o "workspace/annotation/test.record"
!python "workspace/scripts/generate_tfrecord_labelme.py" -j "workspace/images/train/" -i "workspace/images/train/" -l "workspace/annotation/label_map.pbtxt" -o "workspace/annotation/train.record"


Successfully created the TFRecord file: workspace/annotation/test.record
Successfully created the TFRecord file: workspace/annotation/train.record


In [14]:
files

{'TF_RECORD_SCRIPTS': WindowsPath('workspace/scripts/generate_tfrecord_labelme.py'),
 'LABELMAP': WindowsPath('workspace/annotation/label_map.pbtxt'),
 'PIPELINE_PATH': 'workspace\\pretrained_model\\ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8\\pipeline.config',
 'CHECKPOINT_PATH': 'workspace\\pretrained_model\\ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8\\checkpoint'}

In [60]:
!copy {files['PIPELINE_PATH']} {files['CHECKPOINT_PATH'] }

        1 file(s) copied.


In [6]:
files

{'TF_RECORD_SCRIPTS': WindowsPath('workspace/scripts/generate_tfrecord_labelme.py'),
 'LABELMAP': WindowsPath('workspace/annotation/label_map.pbtxt'),
 'PIPELINE_PATH': 'workspace\\pretrained_model\\ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8\\pipeline.config',
 'CHECKPOINT_PATH': 'workspace\\pretrained_model\\ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8\\checkpoint'}

##### 10 UPDATE CONFIG FOR TRANSFER LEARNING

In [5]:
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [7]:
config = config_util.get_configs_from_pipeline_file(files['PIPELINE_PATH'])
config

{'model': ssd {
   num_classes: 2
   image_resizer {
     fixed_shape_resizer {
       height: 320
       width: 320
     }
   }
   feature_extractor {
     type: "ssd_mobilenet_v2_fpn_keras"
     depth_multiplier: 1.0
     min_depth: 16
     conv_hyperparams {
       regularizer {
         l2_regularizer {
           weight: 3.9999998989515007e-05
         }
       }
       initializer {
         random_normal_initializer {
           mean: 0.0
           stddev: 0.009999999776482582
         }
       }
       activation: RELU_6
       batch_norm {
         decay: 0.996999979019165
         scale: true
         epsilon: 0.0010000000474974513
       }
     }
     use_depthwise: true
     override_base_feature_extractor_hyperparams: true
     fpn {
       min_level: 3
       max_level: 7
       additional_layer_depth: 128
     }
   }
   box_coder {
     faster_rcnn_box_coder {
       y_scale: 10.0
       x_scale: 10.0
       height_scale: 5.0
       width_scale: 5.0
     }
   }
   match

In [ ]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(files['PIPELINE_PATH'],'r') as f:
    proto_str = f.read()
    text_format.Merge(proto_str,pipeline_config)

In [ ]:
pipeline_config.model.ssd.num_classes = len(label)
pipeline_config.train_config.batch_size = 4
pipeline_config.train_config.fine_tune_checkpoint = os.path.join(PATHS['pretrained-model'],'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8','checkpoint', 'ckpt-0')
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path= os.path.join(files['LABELMAP'])
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [os.path.join(PATHS['annotation'], 'train.record')]
pipeline_config.eval_input_reader[0].label_map_path = os.path.join(files['LABELMAP'])
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [os.path.join(PATHS['annotation'], 'test.record')]

In [105]:
config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
with tf.io.gfile.GFile(files['PIPELINE_PATH'], "wb") as f:                                                                                                                                                                                                                     
    f.write(config_text)

##### 11. TRAIN THE MODEL

In [106]:
TRAINING_SCRIPT = os.path.join('tfod', 'research', 'object_detection', 'model_main_tf2.py')
# command = "python {} --model_dir={} --pipeline_config_path={} --num_train_steps=2000".format(TRAINING_SCRIPT, PATH['CHECKPOINT_PATH'],file['PIPELING_PATH'])

!python "{TRAINING_SCRIPT}" --model_dir={PATHS['CHECKPOINT_PATH']} --pipeline_config_path={files['PIPELINE_PATH']} --num_train_steps=2000

c:\Users\bkj\AppData\Local\Programs\Python\Python310\lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
W1121 23:53:13.191323 11764 cross_device_ops.py:1387] There are non-GPU devices in `tf.distribute.Strategy`, not using nccl allreduce.
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
I1121 23:53:13.299425 11764 mirrored_strategy.py:374] Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
INFO:tensorflow:Maybe overwriting train_steps: 2000
I1121 23:53:13.321804 1

I1122 00:06:52.391807 11764 model_lib_v2.py:708] {'Loss/classification_loss': 0.17535694,
 'Loss/localization_loss': 0.17041829,
 'Loss/regularization_loss': 0.15440086,
 'Loss/total_loss': 0.5001761,
 'learning_rate': 0.0426662}
INFO:tensorflow:Step 400 per-step time 2.082s
I1122 00:10:20.713449 11764 model_lib_v2.py:705] Step 400 per-step time 2.082s
INFO:tensorflow:{'Loss/classification_loss': 0.15084104,
 'Loss/localization_loss': 0.23577209,
 'Loss/regularization_loss': 0.15417336,
 'Loss/total_loss': 0.5407865,
 'learning_rate': 0.047999598}
I1122 00:10:20.719459 11764 model_lib_v2.py:708] {'Loss/classification_loss': 0.15084104,
 'Loss/localization_loss': 0.23577209,
 'Loss/regularization_loss': 0.15417336,
 'Loss/total_loss': 0.5407865,
 'learning_rate': 0.047999598}
INFO:tensorflow:Step 500 per-step time 2.091s
I1122 00:13:49.765463 11764 model_lib_v2.py:705] Step 500 per-step time 2.091s
INFO:tensorflow:{'Loss/classification_loss': 0.09871819,
 'Loss/localization_loss': 0.141

##### 12. EVALUATE THE MODEL

In [127]:
!python "tfod\\research\\object_detection\\model_main_tf2.py" --model_dir={PATHS['CHECKPOINT_PATH']} --pipeline_config_path="workspace\\pretrained_model\\ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8\\pipeline.config" --checkpoint_dir = {PATHS['CHECKPOINT_PATH']}

c:\Users\bkj\AppData\Local\Programs\Python\Python310\lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
W1122 01:16:52.848561 24040 model_lib_v2.py:1089] Forced number of epochs for all eval validations to be 1.
INFO:tensorflow:Maybe overwriting sample_1_of_n_eval_examples: None
I1122 01:16:52.848561 24040 config_util.py:552] Maybe overwriting sample_1_of_n_eval_examples: None
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I1122 01:16:52.848561 24040 config_util.py:552] Maybe overwriting use_bfloat16: False
INFO:tensorflow:Maybe overwriting ev

#### 13. LOAD TRAIN MODEL FROM CHECKPOINT

In [4]:
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
from object_detection.utils import config_util

c:\Users\bkj\AppData\Local\Programs\Python\Python310\lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [7]:
# load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(files['PIPELINE_PATH'])
detection_model = model_builder.build(model_config=configs['model'],is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(PATHS['CHECKPOINT_PATH'],'ckpt-3')).expect_partial()

@tf.function
def detect_fn(image):
    image,shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image,shapes)
    detections = detection_model.postprocess(prediction_dict,shapes)
    return detections

In [11]:
category_index = label_map_util.create_category_index_from_labelmap(files['LABELMAP'])
IMAGEFILE_PATH = os.path.join(PATHS['test-images'],'2b11bcd1-881e-11ee-bffd-ec5c68664d70.jpg')

In [12]:

mp_holistic = mp.solutions.holistic
holistic = mp_holistic.Holistic()
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

def get_angels(results):
    # Calculate the wrist
        wrist = results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_WRIST]
        index = results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_INDEX]
        elbow = results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_ELBOW]
        
        wrist = np.array([wrist.x, wrist.y, wrist.z])
        index = np.array([index.x, index.y, index.z])
        elbow = np.array([elbow.x, elbow.y, elbow.z])

        v1 = index - wrist
        v2 = elbow - wrist
        wrist_angle = np.arccos(np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2)))
        wrist_angle = np.degrees(wrist_angle)
        if index[1] < wrist[1]:
          wrist_angle = -wrist_angle


        # Calculate the right elbow angle
        shoulder = results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER]
        elbow = results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_ELBOW]
        wrist = results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_WRIST]
        shoulder = np.array([shoulder.x, shoulder.y, shoulder.z])
        elbow = np.array([elbow.x, elbow.y, elbow.z])
        wrist = np.array([wrist.x, wrist.y, wrist.z])
        v1 = shoulder - elbow
        v2 = wrist - elbow
        elbow_angle = np.arccos(np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2)))
        elbow_angle = np.degrees(elbow_angle)

        # Calculate the right shoulder angle
        hip = results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_HIP]
        hip = np.array([hip.x, hip.y, hip.z])
        v1 = elbow - shoulder
        v2 = hip - shoulder
        shoulder_angle = np.arccos(np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2)))
        shoulder_angle = np.degrees(shoulder_angle)
        
        # Calculate the hip angle
        knee = results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_KNEE]
        knee = np.array([knee.x, knee.y, knee.z])
        v1 = shoulder - hip
        v2 = knee - hip
        hip_angle = np.arccos(np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2)))
        hip_angle = np.degrees(hip_angle)

        # Calculate the right knee angle
        ankle = results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_ANKLE]
        ankle = np.array([ankle.x, ankle.y, ankle.z])
        v1 = hip - knee
        v2 = ankle - knee
        knee_angle = np.arccos(np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2)))
        knee_angle = np.degrees(knee_angle)

        # Calculate the right ankle angle
        heel = results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_HEEL]
        heel = np.array([heel.x, heel.y, heel.z])
        v1 = knee - ankle
        v2 = heel - ankle
        ankle_angle = np.arccos(np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2)))
        ankle_angle = np.degrees(ankle_angle)

        return [wrist_angle,elbow_angle,shoulder_angle,hip_angle,knee_angle,ankle_angle]


def display_angle_table(frame,results):
    
    wrist_angle,elbow_angle,shoulder_angle,hip_angle,knee_angle,ankle_angle = get_angels(results)

    cv2.rectangle(frame, (width - 600, height - 250), (width, height), (0, 0, 0), cv2.FILLED)
    cv2.line(frame,(width - 600, height - 250), (width - 600, height ), (255,255,255), 4)
    for i in [250,210,170,130,90,50]:
        cv2.line(frame, (width - 600, height - i), (width, height - i), (255,255,255), 4)
    cv2.putText(frame, f'Right wrist angle: {wrist_angle:.2f} degs', (width - 600, height - 220), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
    cv2.putText(frame, f'Right elbow angle: {elbow_angle:.2f} degs', (width - 600, height - 180), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
    cv2.putText(frame, f'Right shoulder angle: {shoulder_angle:.2f} degs', (width - 600, height - 140), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
    cv2.putText(frame, f'Hip angle: {hip_angle:.2f} degs', (width - 600, height - 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2, cv2.LINE_AA)
    cv2.putText(frame, f'Right knee angle: {knee_angle:.2f} degs', (width - 600, height - 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 255), 2, cv2.LINE_AA)
    cv2.putText(frame, f'Right ankle angle: {ankle_angle:.2f} degs', (width - 600, height - 20), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2, cv2.LINE_AA)


def mediapipe_detection(image,model):
    image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    results = model.process(image)
    image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR)
    return results

def draw_landmarks(image,results):
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(10, 250, 80), thickness=2, circle_radius=4),
                                  mp_drawing.DrawingSpec(color=(250, 0, 0), thickness=2, circle_radius=2),)
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(0, 250, 80), thickness=2, circle_radius=4),
                                  mp_drawing.DrawingSpec(color=(250, 0, 250), thickness=2, circle_radius=2),)
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(10, 250, 80), thickness=2, circle_radius=4),
                                  mp_drawing.DrawingSpec(color=(121, 121, 255), thickness=2, circle_radius=2),)
    

def extract_keypoint(results):
    face = np.array([[res.x,res.y,res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    pose = np.array([[res.x,res.y,res.z,res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    rh = np.array([[res.x,res.y,res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    lh = np.array([[res.x,res.y,res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose,face,lh,rh])

def extract_images(video_path,output_folder,frames_to_extract):
    '''
    extracts images from video
    
    Arguments: 
        video_path: this is the path to the video we are extracting image from
        output_file: this is the directory our extracted image will be saved to
        frame_interval_seconds: this is the seconds interval to save each image
    '''

    cap = cv2.VideoCapture(video_path)
    frame_count = 0
    frame_on_save = 0

    Path(output_folder).mkdir(exist_ok=True)
    folder_name = Path(video_path).stem

    while cap.isOpened():
        ret,frame = cap.read()
        if not ret:
            break

        frame_count += 1
        # current_time_seconds = frame_count / cap.get(cv2.CAP_PROP_FPS)
        # print(current_time_seconds)

        # Save the frame if it's within the desired interval
        if  frame_count in frames_to_extract:

            final_npy_dir = Path(output_folder,folder_name,f"{frame_on_save}.npy")
            image_np = np.array(frame)
        
            input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
            detections = detect_fn(input_tensor)
            
            num_detections = int(detections.pop('num_detections'))
            detections = {key: value[0, :num_detections].numpy()
                            for key, value in detections.items()}
            detections['num_detections'] = num_detections

            # detection_classes should be ints.
            detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

            label_id_offset = 1
            image_np_with_detections = image_np.copy()

            # for i in range(len(detections['detection_boxes'])):
            box = detections['detection_boxes'][0]
            ymin, xmin, ymax, xmax = box
            ymin, xmin, ymax, xmax = int(ymin * height), int(xmin * width), int(ymax * height), int(xmax * width)

            player_frame = image_np[ymin:ymax, xmin:xmax]

            # image,results = mediapipe_detection(player_frame,holistic)

            # print(results)

            # # results_np = extract_keypoint(results)
            # # np.save(results_np,final_npy_dir)
            # # frame_to_save += 1

            output_path = Path(output_folder, f"{str(uuid.uuid1())}.jpg")
            print(output_path)
            cv2.imwrite(str(output_path), player_frame)
    cap.release()

def view_fn(path,starting_point):
    cap = cv2.VideoCapture(path)
    frame_count = 0
    frame_to_save = 0
    folder_name = Path(path).stem
    with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as holistic:
        while cap.isOpened():
            ret,frame = cap.read()
            if not ret:
                break
            
            frame_count += 1
            image_np = np.array(frame)
            
            input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
            detections = detect_fn(input_tensor)
            
            num_detections = int(detections.pop('num_detections'))
            detections = {key: value[0, :num_detections].numpy()
                            for key, value in detections.items()}
            detections['num_detections'] = num_detections

            # detection_classes should be ints.
            detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

            label_id_offset = 1
            image_np_with_detections = image_np.copy()

            # for i in range(len(detections['detection_boxes'])):
            box = detections['detection_boxes'][0]
            ymin, xmin, ymax, xmax = box
            ymin, xmin, ymax, xmax = int(ymin * height), int(xmin * width), int(ymax * height), int(xmax * width)

            player_frame = image_np[ymin:ymax, xmin:xmax]
            
            margin = 50

            text_x = max(min(xmin - margin, player_frame.shape[0] - 150), 0)
            text_y = max(min(ymin - margin, player_frame.shape[1] - 150), 0)

            
            if text_x < player_frame.shape[1] and text_y < player_frame.shape[0]:
                cv2.putText(player_frame, f'{frame_count}', (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
            else:
                print("Text coordinates exceed frame dimensions.")
            cv2.imshow('frame',player_frame)
            

            if frame_count in np.arange(starting_point,starting_point + 25):
                image,results = mediapipe_detection(player_frame,holistic)
                extracted_results = extract_keypoint(results)
                final_npy_dir = Path('shot_detection_images','0',folder_name,f"{frame_to_save}.npy")
                Path('shot_detection_images','0',folder_name).mkdir(exist_ok=True)
                np.save(final_npy_dir,extracted_results)
                draw_landmarks(image,results)
                mp_drawing.draw_landmarks(player_frame, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                                        mp_drawing.DrawingSpec(color=(10, 250, 80), thickness=2, circle_radius=4),
                                        mp_drawing.DrawingSpec(color=(250, 0, 0), thickness=2, circle_radius=2),)
                frame_to_save += 1
            elif frame_count > starting_point + 25:
                break

        cap.release()
        cv2.destroyAllWindows()

        

In [13]:

cap = cv2.VideoCapture('videos/JordanWalker5Shots.MOV')
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
frame_count = 0

mp_pose = mp.solutions.pose
shot_num = 0
with mp_pose.Pose() as pose:
  while cap.isOpened(): 
      ret, frame = cap.read()

      frame_count += 1
      image_np = np.array(frame)
      fps = cap.get(cv2.CAP_PROP_FPS)

      # Indicate frame number and fps at the top-left corner
      cv2.putText(image_np, f'Frame: {frame_count}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
      cv2.putText(image_np, f'FPS: {fps:.2f}', (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
      cv2.putText(image_np, f'Shot number: {shot_num}', (10, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

      
      input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
      detections = detect_fn(input_tensor)
      num_detections = int(detections.pop('num_detections'))
      detections = {key: value[0, :num_detections].numpy()
                    for key, value in detections.items()}
      detections['num_detections'] = num_detections

      # detection_classes should be ints.
      detections['detection_classes'] = detections['detection_classes'].astype(np.int64)
      label_id_offset = 1
      image_np_with_detections = image_np.copy()

      box = detections['detection_boxes'][0]
      ymin, xmin, ymax, xmax = box
      ymin, xmin, ymax, xmax = int(ymin * height), int(xmin * width), int(ymax * height), int(xmax * width)

      cv2.rectangle(image_np_with_detections, (xmin, ymin), (xmax, ymax), (0, 255, 0), 4)

      # Crop the player region and extract mediapip keypoints
      player_frame = image_np[ymin:ymax, xmin:xmax]
      results = mediapipe_detection(player_frame,holistic)
      keypoints = extract_keypoint(results)
  
      # draw landmarks on keypoints and display angles on screen
      draw_landmarks(player_frame,results)
      if results.pose_landmarks:
         display_angle_table(image_np_with_detections,results)

      image_np_with_detections[ymin:ymax, xmin:xmax] = player_frame
      cv2.imshow('object detection', cv2.resize(image_np_with_detections, (1200, 600)))

      
      if cv2.waitKey(1) & 0xFF == ord('q'):
          cap.release()
          cv2.destroyAllWindows()
          break

In [16]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils
mp_drawing_style = mp.solutions.drawing_styles

In [ ]:
mp_holistic.POSE_CONNECTIONS

In [20]:
cap = cv2.VideoCapture('videos/SteveSir5Shots.mov')
frame_count = 0
with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret,frame = cap.read()
        if not ret:
            break
        
        frame_count += 1
        image_np = np.array(frame)

        height, width, _ = image_np.shape
        
        input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
        detections = detect_fn(input_tensor)
        
        num_detections = int(detections.pop('num_detections'))
        detections = {key: value[0, :num_detections].numpy()
                        for key, value in detections.items()}
        detections['num_detections'] = num_detections

        # detection_classes should be ints.
        detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

        label_id_offset = 1
        image_np_with_detections = image_np.copy()

        # for i in range(len(detections['detection_boxes'])):
        box = detections['detection_boxes'][0]
        ymin, xmin, ymax, xmax = box
        ymin, xmin, ymax, xmax = int(ymin * height), int(xmin * width), int(ymax * height), int(xmax * width)

        player_frame = image_np[ymin:ymax, xmin:xmax]


        image,results = mediapipe_detection(player_frame,holistic)
        draw_landmarks(image,results)
        mp_drawing.draw_landmarks(player_frame, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(10, 250, 80), thickness=2, circle_radius=4),
                                  mp_drawing.DrawingSpec(color=(250, 0, 0), thickness=2, circle_radius=2),)

        margin = 50

        text_x = max(min(xmin - margin, player_frame.shape[0] - 150), 0)
        text_y = max(min(ymin - margin, player_frame.shape[1] - 150), 0)

        
        if text_x < player_frame.shape[1] and text_y < player_frame.shape[0]:
            cv2.putText(player_frame, f'{frame_count}', (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
        else:
            print("Text coordinates exceed frame dimensions.")
        cv2.imshow('frame',player_frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

TypeError: cannot unpack non-iterable type object

In [ ]:
extract_images('videos/SteveSir5Shots.mov','shot_detection_images/',np.arange(158,170))

In [37]:
view_fn('videos/CharlieBrownDelaware5shots.mov',20)
view_fn('videos/CoreyHawkins5Shots.mov',20)
view_fn('videos/DavonteFitzgerald5Shots.mov',20)
view_fn('videos/Elfrid5Shots.mov',20)
view_fn('videos/Facu5shots.mov',20)
view_fn('videos/GregBrown5Shots.mov',20)
view_fn('videos/JaredBrownrigdge5shots.mov',20)
view_fn('videos/JordanWalker5Shots.MOV',20)
view_fn('videos/KalleFinland5Shots.mov',20)
view_fn('videos/KylorKelley5Shots.mov',20)
view_fn('videos/Maxi5Shot.mov',20)
view_fn('videos/OMax5Shot.mov',270)
view_fn('videos/SergeIbaka x5.mov',20)
view_fn('videos/SteveSir5Shots.mov',20)


In [58]:
actions = ['noshot','shot']
label_map = {label:num for num,label in enumerate(actions)}
label_map

{'noshot': 0, 'shot': 1}

In [59]:
sequences,labels = [],[]
for action in actions:
    print(action)
    action_path = Path('shot_detection_images',action)
    action_files = [x for x in action_path.glob('*')]
    for player_file in action_files:
        window = []
        if not player_file.is_dir():
            continue
        for i in range(25):
            image_array = np.load(str(Path(player_file,f"{i}.npy")))
            window.append(image_array)
        sequences.append(window)
        labels.append(label_map[action])
        

noshot


shot
